In [1]:
import numpy as np
import pandas as pd
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('data_files/cause_of_deaths.csv')
# print(df.head(10))
print(df.describe(include='all'))


       Country/Territory  Code         Year    Meningitis  \
count               6120  6120  6120.000000   6120.000000   
unique               204   204          NaN           NaN   
top          Afghanistan   AFG          NaN           NaN   
freq                  30    30          NaN           NaN   
mean                 NaN   NaN  2004.500000   1719.701307   
std                  NaN   NaN     8.656149   6672.006930   
min                  NaN   NaN  1990.000000      0.000000   
25%                  NaN   NaN  1997.000000     15.000000   
50%                  NaN   NaN  2004.500000    109.000000   
75%                  NaN   NaN  2012.000000    847.250000   
max                  NaN   NaN  2019.000000  98358.000000   

        Alzheimer's Disease and Other Dementias  Parkinson's Disease  \
count                               6120.000000          6120.000000   
unique                                      NaN                  NaN   
top                                         NaN    

In [33]:
# Save list of column in variable to view
columns = list(df.columns)
print(columns)
# create variable to see all the unique countries that are represented.
countries = df['country'].unique()
# Print the number of unique countries
print(df['country'].nunique())
# rename specific columns that have unusual charcters or redundunt naming convention or commas.
df.rename(columns= {'Country/Territory': 'country', 'Code' : 'country_code', 'Protein-Energy Malnutrition': 'protein_malnutrition', 'Alzheimer\'s Disease and Other Dementias' : 'dementia', 'Self-harm': 'self_harm', 'Fire, Heat, and Hot Substances': 'fire_heat'}, inplace=True)


['country', 'country_code', 'year', 'meningitis', 'dementia', "parkinson's_disease", 'nutritional_deficiencies', 'malaria', 'drowning', 'interpersonal_violence', 'maternal_disorders', 'hiv/aids', 'drug_use_disorders', 'tuberculosis', 'cardiovascular_diseases', 'lower_respiratory_infections', 'neonatal_disorders', 'alcohol_use_disorders', 'self_harm', 'exposure_to_forces_of_nature', 'diarrheal_diseases', 'environmental_heat_and_cold_exposure', 'neoplasms', 'conflict_and_terrorism', 'diabetes_mellitus', 'chronic_kidney_disease', 'poisonings', 'protein_malnutrition', 'road_injuries', 'chronic_respiratory_diseases', 'cirrhosis_and_other_chronic_liver_diseases', 'digestive_diseases', 'fire_heat', 'acute_hepatitis']
204


Edit all column names - change all to lower and replace spaces with underscores.

In [32]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
print(df.columns)

Index(['country', 'country_code', 'year', 'meningitis', 'dementia',
       'parkinson's_disease', 'nutritional_deficiencies', 'malaria',
       'drowning', 'interpersonal_violence', 'maternal_disorders', 'hiv/aids',
       'drug_use_disorders', 'tuberculosis', 'cardiovascular_diseases',
       'lower_respiratory_infections', 'neonatal_disorders',
       'alcohol_use_disorders', 'self_harm', 'exposure_to_forces_of_nature',
       'diarrheal_diseases', 'environmental_heat_and_cold_exposure',
       'neoplasms', 'conflict_and_terrorism', 'diabetes_mellitus',
       'chronic_kidney_disease', 'poisonings', 'protein_malnutrition',
       'road_injuries', 'chronic_respiratory_diseases',
       'cirrhosis_and_other_chronic_liver_diseases', 'digestive_diseases',
       'fire_heat', 'acute_hepatitis'],
      dtype='object')


See how many years are represented in the data. Check data type for each column.

In [42]:
years = df.year.unique()
print(years)
print(df.info())

[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6120 entries, 0 to 6119
Data columns (total 34 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   country                                     6120 non-null   object
 1   country_code                                6120 non-null   object
 2   year                                        6120 non-null   int64 
 3   meningitis                                  6120 non-null   int64 
 4   dementia                                    6120 non-null   int64 
 5   parkinson's_disease                         6120 non-null   int64 
 6   nutritional_deficiencies                    6120 non-null   int64 
 7   malaria                                     6120 non-null   int64 
 8   drowning      

Check for null values in each column.